# Load necessary packages

In [1]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

In [2]:
! pip install 'git+git://github.com/HR/github-clone#egg=ghclone' &> /dev/null

! ghclone https://github.com/timovijn/ElectricityPriceForecasting/tree/master/LSTM

zsh:1: command not found: ghclone


In [3]:
features = pd.read_pickle(f"./features.pkl")

ID3 = pd.DataFrame()

ID3['ID3'] = features['ID3']

In [4]:
features.head(5)

,ID3,VOL,MCP,LOAD,LOAD_F,LOAD_FE,ID3 (-4),ID3 (-5),ID3 (-6),ID3 (-7),...,HOD 14,HOD 15,HOD 16,HOD 17,HOD 18,HOD 19,HOD 20,HOD 21,HOD 22,HOD 23
2015-01-08 01:00:00+00:00,22.953776,439.5,32.32,9008.00,8505.25,502.75,29.934792,61.666667,61.118812,61.370370,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 02:00:00+00:00,23.168355,261.5,31.10,8889.25,8222.25,667.00,29.853669,29.934792,61.666667,61.118812,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 03:00:00+00:00,21.000000,420.5,30.17,8929.25,8122.25,807.00,24.012378,29.853669,29.934792,61.666667,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 04:00:00+00:00,30.000000,460.6,24.54,9423.75,8323.50,1100.25,23.269810,24.012378,29.853669,29.934792,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 05:00:00+00:00,30.000000,250.0,32.00,10884.50,9015.00,1869.50,22.953776,23.269810,24.012378,29.853669,...,0,0,0,0,0,0,0,0,0,0


In [5]:
features.tail(5)

,ID3,VOL,MCP,LOAD,LOAD_F,LOAD_FE,ID3 (-4),ID3 (-5),ID3 (-6),ID3 (-7),...,HOD 14,HOD 15,HOD 16,HOD 17,HOD 18,HOD 19,HOD 20,HOD 21,HOD 22,HOD 23
2018-12-30 14:00:00+00:00,53.790740,446.6,46.19,13842.50,15329.25,1486.75,76.370821,87.755884,78.709213,52.958116,...,1,0,0,0,0,0,0,0,0,0
2018-12-30 15:00:00+00:00,59.477646,131.6,47.64,14319.25,15644.50,1325.25,63.690401,76.370821,87.755884,78.709213,...,0,1,0,0,0,0,0,0,0,0
2018-12-30 16:00:00+00:00,59.883829,310.1,55.94,15120.75,16285.75,1165.00,56.170316,63.690401,76.370821,87.755884,...,0,0,1,0,0,0,0,0,0,0
2018-12-30 17:00:00+00:00,59.471501,220.9,58.40,14728.75,15555.75,827.00,51.675229,56.170316,63.690401,76.370821,...,0,0,0,1,0,0,0,0,0,0
2018-12-30 18:00:00+00:00,58.448950,288.1,69.48,14296.50,14992.25,695.75,53.790740,51.675229,56.170316,63.690401,...,0,0,0,0,1,0,0,0,0,0


# Alternative (2) create train/test data

In [6]:
# def ts_train_test(ID3, time_steps, for_periods):

#     # Create training and test sets
#     mask = (ID3.index.year < 2018)
#     ts_train = ID3.loc[mask]['ID3'].values
#     ts_train = ts_train.reshape(1, -1).transpose()

#     mask = (ID3.index.year == 2018)
#     ts_test = ID3.loc[mask]['ID3'].values
#     ts_test = ts_test.reshape(1, -1).transpose()

#     # ts_train = ID3[:'2018'].iloc['ID3'].values
#     # ts_test  = ID3['2019':].iloc['ID3'].values

#     # Scale the data
#     from sklearn.preprocessing import MinMaxScaler
#     sc = MinMaxScaler(feature_range=(0,1))
#     ts_train_unscaled = ts_train
#     ts_train = sc.fit_transform(ts_train)

#     # Create training data of s samples and t time steps
#     X_train = []
#     y_train = []
#     y_train_stacked = []
    
#     for i in range(time_steps, len(ts_train)-1): 
#         X_train.append(ts_train[i-time_steps:i,0])
#         y_train.append(ts_train[i:i+for_periods,0])
#     X_train, y_train = np.array(X_train), np.array(y_train)

#     # Reshape X_train for efficient modelling
#     X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

#     # Prepare to create X_test
#     inputs = pd.concat((ID3['ID3'][:'2018'], ID3['ID3']['2019':]),axis=0).values
#     inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
#     inputs = inputs.reshape(-1,1)
#     inputs  = sc.transform(inputs)

#     X_test = []
#     for i in range(time_steps, len(ts_test)+time_steps-for_periods):
#         X_test.append(inputs[i-time_steps:i,0])
        
#     X_test = np.array(X_test)
#     X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

#     return X_train, y_train , X_test, sc

# ##########################################################################
# X_train, y_train, X_test, sc = ts_train_test(ID3, 5, 2)
# ##########################################################################

# X_train.shape[0], X_train.shape[1]

# print(X_train)

# Create train/test data

In [7]:
time_steps = 24
for_periods = 1
lag = 23                 # Extra lag

# Create training and test sets
mask = (ID3.index.year < 2018)
train_mat = ID3.loc[mask]['ID3']
ts_train = np.array(ID3.loc[mask]['ID3'])
ts_train = ts_train.reshape(1, -1).transpose()

mask = (ID3.index.year == 2018)
test_mat = ID3.loc[mask]['ID3']
ts_test = np.array(ID3.loc[mask]['ID3'])
ts_test = ts_test.reshape(1, -1).transpose()

# # Scale the data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))

# Fit scaler on train and transform train

ts_train_unscaled = ts_train
ts_train = sc.fit_transform(ts_train)

# Transform test with train-fitted scaler

ts_test_unscaled = ts_test
ts_test = sc.transform(ts_test)

# Create training data of s samples and t time steps
X_train = []
y_train = []

for i in range(time_steps+lag, len(ts_train)):
    X_train.append(ts_train[i-lag-time_steps:i-lag,0])
    y_train.append(ts_train[i:i+for_periods,0])

X_train = np.array(X_train)
y_train = np.array([item for sublist in y_train for item in sublist])
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_test = []
y_test = []
y_test_unscaled = []

for i in range(time_steps+lag, len(ts_test)):
    X_test.append(ts_test[i-lag-time_steps:i-lag,0])
    y_test.append(ts_test[i:i+for_periods,0])
    y_test_unscaled.append(ts_test_unscaled[i:i+for_periods,0])

X_test = np.array(X_test)
y_test = np.array([item for sublist in y_test for item in sublist])
y_test_unscaled = np.array([item for sublist in y_test_unscaled for item in sublist])
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [8]:
print(train_mat), print(), print(train_mat.shape);

2015-01-08 01:00:00+00:00    22.953776
2015-01-08 02:00:00+00:00    23.168355
2015-01-08 03:00:00+00:00    21.000000
2015-01-08 04:00:00+00:00    30.000000
2015-01-08 05:00:00+00:00    30.000000
                               ...    
2017-12-31 19:00:00+00:00    80.726384
2017-12-31 20:00:00+00:00    67.293939
2017-12-31 21:00:00+00:00    24.224867
2017-12-31 22:00:00+00:00    18.912550
2017-12-31 23:00:00+00:00    16.199165
Name: ID3, Length: 25222, dtype: float64

(25222,)


In [9]:
print(test_mat), print(), print(test_mat.shape);

2018-01-01 00:00:00+00:00    14.586875
2018-01-01 01:00:00+00:00    12.990924
2018-01-01 02:00:00+00:00    22.150235
2018-01-01 03:00:00+00:00    21.917514
2018-01-01 04:00:00+00:00    21.621781
                               ...    
2018-12-30 14:00:00+00:00    53.790740
2018-12-30 15:00:00+00:00    59.477646
2018-12-30 16:00:00+00:00    59.883829
2018-12-30 17:00:00+00:00    59.471501
2018-12-30 18:00:00+00:00    58.448950
Name: ID3, Length: 8623, dtype: float64

(8623,)


# Alternative create train/test

In [10]:
# inputs = features[['ID3']]
# outputs = features[['ID3']]

# inputs_train = inputs[inputs.index.year < 2018]
# outputs_train = outputs[outputs.index.year < 2018]

# inputs_test = inputs[inputs.index.year == 2018]
# outputs_test = outputs[outputs.index.year == 2018]

# # Scale inputs
# scaler = MinMaxScaler()

# inputs_train = pd.DataFrame(scaler.fit_transform(inputs_train), columns = inputs_train.columns)
# inputs_test = pd.DataFrame(scaler.transform(inputs_test), columns = inputs_test.columns)

# # Scale outputs
# sc = MinMaxScaler()

# sc.fit_transform(outputs_train)

# outputs_train = pd.DataFrame(sc.fit_transform(outputs_train), columns = outputs_train.columns)
# outputs_test = pd.DataFrame(sc.transform(outputs_test), columns = outputs_test.columns)

# time_steps = 6
# for_periods = 1
# lag = 4

# # X_train and y_train

# X_train = []
# y_train = []

# for i in range(0, len(outputs_train)-time_steps-lag+1):
#     for ii in range(0, time_steps):
#         X_train.extend(inputs_train.iloc[i+ii].to_numpy())
#     y_train.extend(outputs_train.iloc[i+time_steps+lag-1].to_numpy())

# X_train, y_train = np.array(X_train), np.array(y_train)

# X_train = X_train.reshape(y_train.shape[0], time_steps, inputs.shape[1])

# # X_test and y_test

# X_test = []
# y_test = []

# for i in range(0, len(outputs_test)-time_steps-lag+1):
#     for ii in range(0, time_steps):
#         X_test.extend(inputs_test.iloc[i+ii].to_numpy())
#     y_test.extend(outputs_test.iloc[i+time_steps+lag-1].to_numpy())

# X_test, y_test = np.array(X_test), np.array(y_test)

# sc = MinMaxScaler(feature_range=(0,1))

# X_test = X_test.reshape(y_test.shape[0], time_steps, inputs.shape[1])

# Create see data

In [11]:
# Convert the 3-D shape of X_train to a data frame so we can see: 
X_train_see = pd.DataFrame(np.reshape(X_train, (X_train.shape[0], X_train.shape[1])))
y_train_see = pd.DataFrame(y_train)

# Convert the 3-D shape of X_test to a data frame so we can see: 
X_test_see = pd.DataFrame(np.reshape(X_test, (X_test.shape[0], X_test.shape[1])))
y_test_see = pd.DataFrame(y_test)
y_test_unscaled_see = pd.DataFrame(y_test_unscaled)

# See train data

In [12]:
print(), print(colored(f'There are {X_train.shape[0]} samples in the training data', 'green'));

print(), print(X_train.shape), print(), print(X_train_see);
print(), print(y_train.shape), print(), print(y_train_see);


There are 25175 samples in the training data

(25175, 24, 1)

             0         1         2         3         4         5         6   \
0      0.099373  0.099945  0.094165  0.118155  0.118155  0.153218  0.154377   
1      0.099945  0.094165  0.118155  0.118155  0.153218  0.154377  0.154241   
2      0.094165  0.118155  0.118155  0.153218  0.154377  0.154241  0.166808   
3      0.118155  0.118155  0.153218  0.154377  0.154241  0.166808  0.167958   
4      0.118155  0.153218  0.154377  0.154241  0.166808  0.167958  0.162353   
...         ...       ...       ...       ...       ...       ...       ...   
25170  0.143479  0.129030  0.117914  0.110427  0.112221  0.113326  0.119709   
25171  0.129030  0.117914  0.110427  0.112221  0.113326  0.119709  0.121066   
25172  0.117914  0.110427  0.112221  0.113326  0.119709  0.121066  0.123108   
25173  0.110427  0.112221  0.113326  0.119709  0.121066  0.123108  0.126596   
25174  0.112221  0.113326  0.119709  0.121066  0.123108  0.126596  0

# See test data

In [13]:
print(), print(colored(f'There are {X_test.shape[0]} samples in the test data', 'green')), print();

print(), print(X_test.shape), print(), print(X_test_see);
print(), print(y_test_unscaled.shape), print(), print(y_test_unscaled_see);


There are 8576 samples in the test data


(8576, 24, 1)

            0         1         2         3         4         5         6   \
0     0.077070  0.072816  0.097231  0.096611  0.095822  0.097771  0.099687   
1     0.072816  0.097231  0.096611  0.095822  0.097771  0.099687  0.103083   
2     0.097231  0.096611  0.095822  0.097771  0.099687  0.103083  0.105514   
3     0.096611  0.095822  0.097771  0.099687  0.103083  0.105514  0.105132   
4     0.095822  0.097771  0.099687  0.103083  0.105514  0.105132  0.104476   
...        ...       ...       ...       ...       ...       ...       ...   
8571  0.219027  0.216908  0.213517  0.207682  0.185106  0.189375  0.185842   
8572  0.216908  0.213517  0.207682  0.185106  0.189375  0.185842  0.189481   
8573  0.213517  0.207682  0.185106  0.189375  0.185842  0.189481  0.186333   
8574  0.207682  0.185106  0.189375  0.185842  0.189481  0.186333  0.185471   
8575  0.185106  0.189375  0.185842  0.189481  0.186333  0.185471  0.186506   

     

In [14]:
import keras
import keras.backend as K

class Dropout(keras.layers.Dropout):
  """Applies Dropout to the input.
  The Dropout layer randomly sets input units to 0 with a frequency of `rate`
  at each step during training time, which helps prevent overfitting.
  Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over
  all inputs is unchanged.
  Note that the Dropout layer only applies when `training` is set to True
  such that no values are dropped during inference. When using `model.fit`,
  `training` will be appropriately set to True automatically, and in other
  contexts, you can set the kwarg explicitly to True when calling the layer.
  (This is in contrast to setting `trainable=False` for a Dropout layer.
  `trainable` does not affect the layer's behavior, as Dropout does
  not have any variables/weights that can be frozen during training.)
  >>> tf.random.set_seed(0)
  >>> layer = tf.keras.layers.Dropout(.2, input_shape=(2,))
  >>> data = np.arange(10).reshape(5, 2).astype(np.float32)
  >>> print(data)
  [[0. 1.]
   [2. 3.]
   [4. 5.]
   [6. 7.]
   [8. 9.]]
  >>> outputs = layer(data, training=True)
  >>> print(outputs)
  tf.Tensor(
  [[ 0.    1.25]
   [ 2.5   3.75]
   [ 5.    6.25]
   [ 7.5   8.75]
   [10.    0.  ]], shape=(5, 2), dtype=float32)
  Args:
    rate: Float between 0 and 1. Fraction of the input units to drop.
    noise_shape: 1D integer tensor representing the shape of the
      binary dropout mask that will be multiplied with the input.
      For instance, if your inputs have shape
      `(batch_size, timesteps, features)` and
      you want the dropout mask to be the same for all timesteps,
      you can use `noise_shape=(batch_size, 1, features)`.
    seed: A Python integer to use as random seed.
  Call arguments:
    inputs: Input tensor (of any rank).
    training: Python boolean indicating whether the layer should behave in
      training mode (adding dropout) or in inference mode (doing nothing).
  """
  
  def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
    super(Dropout, self).__init__(rate, noise_shape=None, seed=None, **kwargs)
    self.training = training

  def call(self, inputs, training=None):
    if 0. < self.rate < 1.:
        noise_shape = self._get_noise_shape(inputs)

        def dropped_inputs():
            return K.dropout(inputs, self.rate, noise_shape, seed=self.seed)

        if not training:
            return K.in_train_phase(dropped_inputs, inputs, training=self.training)
        return K.in_train_phase(dropped_inputs, inputs, training=training)
    return inputs

In [15]:
def LSTM_model(X_train, y_train, X_test, sc):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, GRU, LSTM
    from keras.optimizers import SGD, Adam
    
    model_lstm = Sequential()
    model_lstm.add(LSTM(units = 200, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
    model_lstm.add(LSTM(units = 100, return_sequences = True))
    model_lstm.add(LSTM(units = 50, return_sequences = False))
    
    model_lstm.add(Dense(units = 1, activation='linear'))

    optimizer = Adam(clipvalue=0.5)
    # model_lstm.compile(loss = 'mse', optimizer=optimizer)

    # Compiling
    model_lstm.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    model_lstm.fit(X_train,y_train,epochs=1,batch_size=32, verbose=1)

    LSTM_prediction = model_lstm.predict(X_test)
    LSTM_prediction = sc.inverse_transform(LSTM_prediction)

    # self.model = model_lstm

    return model_lstm, LSTM_prediction

def LSTM_model2(X_train, y_train, X_test, sc):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, GRU, LSTM
    from keras.optimizers import SGD, Adam
    
    model_lstm = Sequential()
    
    model_lstm.add(LSTM(units = 200, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
    
    model_lstm.add(Dropout(0.2, training=True))
    
    model_lstm.add(LSTM(units = 100, return_sequences = False))

    model_lstm.add(Dropout(0.2, training=True))
    
    # model_lstm.add(LSTM(units = 50, return_sequences = False))
    
    model_lstm.add(Dense(units = 1, activation='relu'))

    optimizer = Adam(clipvalue=0.5)
    # model_lstm.compile(loss = 'mse', optimizer=optimizer)

    # Compiling
    model_lstm.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    model_lstm.fit(X_train,y_train,epochs=5,batch_size=32, verbose=1)

    LSTM_prediction = model_lstm.predict(X_test)
    LSTM_prediction = sc.inverse_transform(LSTM_prediction)

    # self.model = model_lstm

    return model_lstm, LSTM_prediction

my_LSTM_model, LSTM_prediction = LSTM_model2(X_train, y_train, X_test, sc)

Epoch 1/5
787/787 [==============================] - 63s 76ms/step - loss: 0.0018
Epoch 2/5
787/787 [==============================] - 54s 69ms/step - loss: 0.0015
Epoch 3/5
787/787 [==============================] - 70s 89ms/step - loss: 0.0014
Epoch 4/5
787/787 [==============================] - 64s 81ms/step - loss: 0.0014
Epoch 5/5
787/787 [==============================] - 63s 81ms/step - loss: 0.0014


In [16]:
from keras.utils.vis_utils import plot_model
plot_model(my_LSTM_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [17]:
import plotly.express as px
import plotly.graph_objects as go

evaluation = pd.DataFrame()
evaluation['Actual'] = y_test_unscaled
evaluation['Predicted'] = LSTM_prediction

fig1 = go.Scatter(      x = evaluation.index,
                        y = evaluation['Actual'],
                        name = 'Actual'
                        # color = evaluation['Type'],
                        # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

fig2 = go.Scatter(      x = evaluation.index,
                        y = evaluation['Predicted'],
                        name = 'Predicted'
                        # color = evaluation['Type'],
                        # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

data = [fig1, fig2]

fig = go.Figure(data=data)

fig.show()

In [18]:
# def actual_pred_plot(preds):

#     actual_pred = pd.DataFrame()

#     mask = (ID3.index.year == 2018)
#     actual_pred['Actual'] = ID3.loc[mask]['ID3'][0:len(preds)]
#     actual_pred['Predicted'] = preds[:, 0]

#     from keras.metrics import MeanSquaredError
#     m = MeanSquaredError()
#     actual_pred['Actual'] = actual_pred['Actual']
#     actual_pred['Predicted'] = actual_pred['Predicted']
#     m.update_state(np.array(actual_pred['Actual']), np.array(actual_pred['Predicted']))
    
#     return (m.result().numpy(), actual_pred)

# m, prediction = actual_pred_plot(LSTM_prediction)

In [19]:
# X = features.drop(labels=['ID3', 'VOL', 'LOAD_F', 'LOAD_FE'], axis=1)
# y = features['ID3']

# X_train, X_test, y_train, y_test = train_test_split(
#     X,
#     y,
#     test_size = 0.3,
#     random_state = 0,
#     shuffle = True)

# X_train.shape, X_test.shape

In [20]:
# X_train_unscaled = X_train
# X_test_unscaled = X_test

# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train_unscaled)

# X_test = scaler.transform(X_test_unscaled)

In [21]:
# def MLP_model(X_train, y_train, X_test):
#     # create a model
#     from keras.models import Sequential
#     from keras.layers import Dense
#     from keras.optimizers import SGD, Adam
    
#     model_MLP = Sequential()
#     model_MLP.add(Dense(200, activation='relu', kernel_initializer='he_normal', input_shape=(X_train.shape[1],)))
#     model_MLP.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
#     model_MLP.add(Dense(1))

#     optimizer = Adam(clipvalue=0.5)
#     # model_lstm.compile(loss = 'mse', optimizer=optimizer)

#     # Compiling
#     model_MLP.compile(optimizer=optimizer,loss='mean_squared_error')
#     # Fitting to the training set
#     model_MLP.fit(X_train, y_train, epochs = 50, batch_size = 16, verbose = 5)

#     prediction = model_MLP.predict(X_test)
#     # prediction = sc.inverse_transform(prediction)

#     # self.model = model_lstm

#     return model_MLP, prediction

# model, prediction = MLP_model(X_train, y_train, X_test)

In [22]:
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping
# #Step1. Define the model
# model = Sequential()
# model.add(Dense(16, activation = 'relu', kernel_initializer = 'he_normal', input_shape = (X_train.shape[1],)))
# model.add(Dense(8, activation = 'relu', kernel_initializer = 'he_normal'))
# model.add(Dense(1))
# #Step2. Compile the model
# model.compile(optimizer = 'adam', loss = 'mse', metrics = 'mae')
# #Step3. Fit the model
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,batch_size=64, verbose=5)
# #Step4.1 Evaluate the model
# loss, mae = model.evaluate(X_test, y_test)
# #Step4.2 Plot the learning curve
# plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='val')
# plt.show()

# prediction = model.predict(X_test)

In [23]:
# actual_pred = pd.DataFrame()

# actual_pred['Actual'] = features['ID3'][0:len(prediction)]
# actual_pred['Predicted'] = prediction[:, 0]

# from keras.metrics import MeanSquaredError
# m = MeanSquaredError()
# actual_pred['Actual'] = actual_pred['Actual'].astype(int)
# actual_pred['Predicted'] = actual_pred['Predicted'].astype(int)
# m.update_state(np.array(actual_pred['Actual']), np.array(actual_pred['Predicted']))

# x, y = actual_pred.index, actual_pred['Actual']

# fig, ax = plt.subplots(figsize = (5 ,5))
# ax.plot(x, y)

# plt.xticks(rotation = 90)

# x, y = actual_pred.index, actual_pred['Predicted']

# fig, ax = plt.subplots(figsize = (5 ,5))
# ax.plot(x, y)

# plt.xticks(rotation = 90);

# m

In [24]:
# # ####################################################################################################
# # # Pre

# actual_pred = prediction

# # actual_pred = pd.DataFrame()

# # actual_pred['Actual'] = features['ID3'][0:len(prediction)]
# # actual_pred['Predicted'] = prediction[:, 0]

# # ####################################################################################################
# # # Plot 1

# # fig, ax = plt.subplots(figsize = (10 ,5))

# # x, y = actual_pred[0:100].index, actual_pred[0:100]['Actual']

# # ax.plot(x, y, linewidth=0.5)

# # ####################################################################################################
# # # Plot 2

# # x, y = actual_pred[0:100].index, actual_pred[0:100]['Predicted']

# # ax.plot(x, y, alpha = 0.5)

# # plt.xticks(rotation = 90);

# import plotly.express as px
# import plotly.graph_objects as go

# actual = pd.DataFrame(['ID3'])
# predicted = pd.DataFrame(['ID3'])

# actual = pd.DataFrame(actual_pred['Actual'].values)
# actual['Type'] = 'Actual'
# predicted = pd.DataFrame(actual_pred['Predicted'].values)
# predicted['Type'] = 'Predicted'

# evaluation = pd.DataFrame(pd.concat([actual, 
#                         predicted],
#                         axis=0))

# fig1 = go.Scatter(      x = actual_pred.index,
#                     y = actual_pred['Actual'],
#                   name = 'Actual'
#                     # color = evaluation['Type'],
#                     # title = "Log of Appliance Energy Consumption in Wh vs Time"
#                     )

# fig2 = go.Scatter(      x = actual_pred.index,
#                     y = actual_pred['Predicted'],
#                   name = 'Predicted'
#                     # color = evaluation['Type'],
#                     # title = "Log of Appliance Energy Consumption in Wh vs Time"
#                     )

# data = [fig1, fig2]

# fig = go.Figure(data=data)

# fig.show()

In [25]:
n_experiments = 50

test_uncertainty_df = pd.DataFrame()

for i in range(1, n_experiments + 1):
  experiment_prediction = my_LSTM_model.predict(X_test)
  test_uncertainty_df['ID3_{}'.format(i)] = np.concatenate(sc.inverse_transform(experiment_prediction), axis = 0)
  
  if i % 1 == 0:
    print(f'Experiment: {i}/{n_experiments}')

# log_energy_consumption_df = test_uncertainty_df.filter(like='ID3', axis=1)
test_uncertainty_df['ID3_mean'] = test_uncertainty_df.mean(axis=1)
test_uncertainty_df['ID3_std'] = test_uncertainty_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['ID3_mean', 'ID3_std']]

test_uncertainty_df['lower_bound'] = test_uncertainty_df['ID3_mean'] - 3*test_uncertainty_df['ID3_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['ID3_mean'] + 3*test_uncertainty_df['ID3_std']

print(), print(test_uncertainty_df);

Experiment: 1/50
Experiment: 2/50
Experiment: 3/50
Experiment: 4/50
Experiment: 5/50
Experiment: 6/50
Experiment: 7/50
Experiment: 8/50
Experiment: 9/50
Experiment: 10/50
Experiment: 11/50
Experiment: 12/50
Experiment: 13/50
Experiment: 14/50
Experiment: 15/50
Experiment: 16/50
Experiment: 17/50
Experiment: 18/50
Experiment: 19/50
Experiment: 20/50
Experiment: 21/50
Experiment: 22/50
Experiment: 23/50
Experiment: 24/50
Experiment: 25/50
Experiment: 26/50
Experiment: 27/50
Experiment: 28/50
Experiment: 29/50
Experiment: 30/50
Experiment: 31/50
Experiment: 32/50
Experiment: 33/50
Experiment: 34/50
Experiment: 35/50
Experiment: 36/50
Experiment: 37/50
Experiment: 38/50
Experiment: 39/50
Experiment: 40/50
Experiment: 41/50
Experiment: 42/50
Experiment: 43/50
Experiment: 44/50
Experiment: 45/50
Experiment: 46/50
Experiment: 47/50
Experiment: 48/50
Experiment: 49/50
Experiment: 50/50

       ID3_mean   ID3_std  lower_bound  upper_bound
0     28.873573  1.151137    25.420162    32.326984
1   

In [26]:
import plotly.graph_objects as go

upper_trace = go.Scatter(
    y=test_uncertainty_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )

lower_trace = go.Scatter(
    y=test_uncertainty_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.5)',
    name='99% Lower Confidence Bound'
    )

real_trace = go.Scatter(
    y=evaluation['Actual'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

mean_trace = go.Scatter(
    y=test_uncertainty_df['ID3_mean'],
    mode='lines',
    fill=None,
    name='Mean Values'
    )

data = [upper_trace, lower_trace, mean_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for ID3 Test Data',
                   xaxis_title='Timestamp',
                   yaxis_title='ID3 (€)')

fig.layout.font.family = 'SF Mono'

fig.show()

In [27]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_df['ID3_mean']
bounds_df['real_value'] = evaluation['Actual']
bounds_df['upper_bound'] = test_uncertainty_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:", 
      bounds_df['contained'].mean())

Proportion of points contained within 99% confidence interval: 0.3478311567164179


In [28]:
# ####################################################################################################
# # Plot 1

# fig, ax = plt.subplots(figsize = (10 ,5))

# x, y = actual_pred.index, actual_pred['Actual']

# ax.plot(x, y)

In [29]:
# ####################################################################################################
# # Plot 2

# fig, ax = plt.subplots(figsize = (10 ,5))

# x, y = actual_pred.index, actual_pred['Predicted']

# ax.plot(x, y, alpha = 0.5)

In [30]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

smape(evaluation['Actual'], evaluation['Predicted'])

21.109211339159298